In [2]:
import pickle as pkl
import numpy as np

In [3]:
data= pkl.load(open("author_data.p", "rb"))

In [ ]:
try:
    user_id_dict, user_count = pkl.load(open('user_id3.p', 'rb'))
    item_id_dict, item_count = pkl.load(open('item_id3.p', 'rb'))
    id_img_dict = pkl.load(open('id_img3.p', 'rb'))
except FileNotFoundError:
    user_id_dict = {}; item_id_dict = {}; id_img_dict = {}
    user_count = 0; item_count = 0

In [ ]:
try:
    favoriter_dict = pkl.load(open('favorited_dict3.p', 'rb'))
except FileNotFoundError:
    favoriter_dict = {}

In [ ]:
artists = list(data.keys())

In [ ]:
#pkl.dump(artists, open('used_artists.p', 'wb'))

In [ ]:
img_ids = []
favorite_lists = []
expanded_artists = []
times = []

In [ ]:
dd_dict = {}
time_dict = {}
cat_dict = {}

In [4]:
categories = [data[x] for x in data]
categories = [y for x in categories for y in x]
all_categories = [x[4] for x in categories if x is not None]
filtered = [x for x in all_categories if 'digital' in x or 'traditional' in x]

In [5]:
def category_filter(string):
    bad_strings = ['text', 'ascii', 'streetart', 'streetart']
    for bad in bad_strings:
        if bad in string:
            return False   
    return True

filtered2 = [x for x in filtered if category_filter(x)]

In [11]:
np.unique(filtered2)

array(['anthro/digital/3d', 'anthro/digital/drawings',
       'anthro/digital/drawings/animals', 'anthro/digital/drawings/comic',
       'anthro/digital/drawings/fantasy',
       'anthro/digital/drawings/macabre',
       'anthro/digital/drawings/political',
       'anthro/digital/drawings/portraits',
       'anthro/digital/drawings/scifi', 'anthro/digital/mixed',
       'anthro/digital/oekaki', 'anthro/digital/other',
       'anthro/digital/paintings', 'anthro/digital/paintings/animals',
       'anthro/digital/paintings/fantasy',
       'anthro/digital/paintings/macabre',
       'anthro/digital/paintings/portraits',
       'anthro/digital/paintings/sci-fi', 'anthro/digital/pixel',
       'anthro/digital/tutorials', 'anthro/digital/vector',
       'anthro/traditional/drawings',
       'anthro/traditional/drawings/animals',
       'anthro/traditional/drawings/fantasy',
       'anthro/traditional/drawings/portraits',
       'anthro/traditional/mixedmedia', 'anthro/traditional/other',
    

In [ ]:
categories = [data[x] for x in data]
categories = [y for x in categories for y in x]
categories = np.unique([x[4] for x in categories if x is not None])

In [ ]:
first_category = np.unique([x.split('/')[0] for x in categories])

In [ ]:
len(categories)

In [ ]:
first_category
cat_int_dict = dict(zip(first_category, range(0,len(first_category))))

In [ ]:
cat_int_dict

In [ ]:
from datetime import datetime

In [ ]:
datetime.fromtimestamp(1502061606)

In [ ]:
datetime.fromtimestamp(1502061606-36*3600*24-16*3600-20*60-6)

In [ ]:
data[artists[0]][0]

In [ ]:
for artist in artists:
    
    artist_data = data[artist]
    try:
        artist_id = user_id_dict[artist]
    except:
        artist_id = user_count;
        user_id_dict[artist] = artist_id; user_count += 1

        
    for entry in artist_data:
        '''Set item id dict'''
        if not entry: continue
        img_id = entry[1]

        if img_id in item_id_dict: # For some reason, there's a duplicate
            continue # So just keep going 
        else:
            img_ids.append(img_id)

        item_id = item_count;
        item_id_dict[img_id] = item_id; item_count += 1
        
        img_src = entry[3]
    
        id_img_dict[item_id] = (img_src, artist_id)

        favorite_list = entry[7]
        favorite_lists.append(favorite_list)
        for favoriter in favorite_list:
            favoriter = favoriter[0]
            try:
                fav_id = user_id_dict[favoriter]
            except:
                fav_id = user_count;
                user_id_dict[favoriter] = fav_id; user_count += 1
            finally:
                try:
                    favoriter_dict[fav_id].extend([item_id])
                except KeyError:
                    favoriter_dict[fav_id] = [item_id]
        dd_dict[item_id] = entry[8] is not None
        time_dict[item_id] = entry[2] < 1502061606-36*3600*24-16*3600-20*60-6 #After July 1st? 
        expanded_artists.append(artist) #need this for later
        times.append(entry[2])
        
        entry_cat = entry[4]
        entry_first_cat = entry_cat.split('/')[0]
        cat_dict[item_id] = cat_int_dict[entry_first_cat]
                            

In [ ]:
cat_dict

In [ ]:
sum(time_dict.values())/len(time_dict)

In [ ]:
pkl.dump(time_dict, open('time_dict3.p', 'wb'))

In [ ]:
pkl.dump(dd_dict, open('dd_dict3.p', 'wb'))

In [ ]:
pkl.dump(cat_dict, open('cat_dict3.p', 'wb'))

In [ ]:
converted_img_ids = [item_id_dict[item] for item in img_ids]
num_favorites = [len(item) for item in favorite_lists]

img_nfavs_dict = dict(zip(converted_img_ids, num_favorites))
pkl.dump(img_nfavs_dict, open('img_nfavs3.p', 'wb'))

In [ ]:
rev = dict(zip(list(user_id_dict.values()), list(user_id_dict.keys()))) #map user_id to user 

In [ ]:
pkl.dump(rev, open('rev_dict3.p', 'wb'))

In [ ]:
pkl.dump(favoriter_dict, open('favorited_dict3.p', 'wb'))
pkl.dump((user_id_dict, user_count), open('user_id3.p', 'wb'))
pkl.dump((item_id_dict, item_count), open('item_id3.p', 'wb'))
pkl.dump(id_img_dict,  open('id_img3.p', 'wb'))


max_item = max(item_id_dict.values())
max_user = max(user_id_dict.values())
pkl.dump((max_item+1, max_user+1), open('useful_stats3.p', 'wb'))

In [ ]:
g5 = sum(map(lambda x: len(x)>=5, favoriter_dict.values()))
sum_g5 = sum(map(lambda x: len(x) if len(x)>5 else 0, favoriter_dict.values()))
num_users = len(user_id_dict)
num_items = len(item_id_dict)
num_ratings = sum(map(lambda x: len(x), favoriter_dict.values()))

print ("""There are {} users and {} items in the dataset, for a total of {} favorites. 
Of those users, {} of them have favorited more than 5 objects, accounting for {} ratings.""".format(
    num_users, num_items, num_ratings, g5, sum_g5))

In [ ]:
artist_dict = {}
item_to_artist = {}

In [ ]:
for artist, img_id, time in zip(expanded_artists,img_ids, times):
    artist_id = user_id_dict[artist]
    
    item_id = item_id_dict[img_id]
    try:
        artist_dict[artist_id].extend([(item_id, time)])
    except:
        artist_dict[artist_id] = [(item_id, time)]
        
    item_to_artist[item_id] = artist_id

In [ ]:
art_time_dict = {}

In [ ]:
for img_id, time in zip(img_ids, times):
    item_id = item_id_dict[img_id]
    art_time_dict[item_id]=time


In [ ]:
pkl.dump(art_time_dict, open('art_time3.p', 'wb'))

In [ ]:
import operator

for artist in artist_dict:
    artist_dict[artist].sort(key=operator.itemgetter(1))
    artist_dict[artist] = [i[0] for i in artist_dict[artist]]

In [ ]:
g5 = sum(map(lambda x: len(x)>=5, artist_dict.values()))
g20 = sum(map(lambda x: len(x)>=20, artist_dict.values()))
g50 = sum(map(lambda x: len(x)>=50, artist_dict.values()))

total50 = sum(map(lambda x: len(x) if len(x)>=50 else 0, artist_dict.values()))

total = sum(map(lambda x: len(x), artist_dict.values()))

num_artists = len(artist_dict)


print('There are {} total artists, of whom {}, {}, {} have more than 5, 20, 50 respectively'.format(num_artists, g5, g20, g50))
print ('The {} artists with 50+ uploads have posted {} of the {} pieces of artwork'.format(g50, total50, total))

In [ ]:
pkl.dump(artist_dict, open('artist_dict3.p', 'wb'))
pkl.dump(item_to_artist, open('item_to_artist3.p', 'wb'))
pkl.dump(art_time_dict, open('art_time3.p', 'wb'))